# Eisai - Accumulations

In [1]:
import sys
sys.path.insert(0, '../../../lib/')

import tabula
import numpy as np
import pandas as pd
import importlib

import pdfexport
importlib.reload(pdfexport)

from pdfexport import *

In [2]:
# Read pdf into DataFrame
df = tabula.read_pdf("transparencyreport-2018_def.pdf", pages='all', lattice=True)

In [8]:
df_export = df.copy()

#Shift all where column[0] not empty
column0 = df_export[df_export.columns[0]]
df_export[column0.notna()] = df_export[column0.notna()].shift(1, axis='columns')

#Drop Columns
df_export = df_export.drop(df.columns[[0, 8]],axis=1)
df_export = df_export.drop(df.columns[10:],axis=1)

#Rename
df_export.columns = fix_columns_accumulations[0: 8]

# Select
amount = df_export[df_export.type.str.contains('Aggregate amount', na=False)]
recipients = df_export[df_export.type.str.contains('Number of', na=False)]
rnd = df_export[df_export.type.str.contains('Transfers of Value re Research', na=False)]

df_export = pd.concat([amount, recipients, rnd])

#Rename first column
df_export = df_export.reset_index(drop=True)
df_export.iloc[0,0] = "hcp_amount"
df_export.iloc[1,0] = "hco_amount"
df_export.iloc[2,0] = "hcp_count"
df_export.iloc[3,0] = "hco_count"
df_export.iloc[4,0] = "rnd"

#copy RnD to total
df_export.loc[df_export.type == 'rnd', 'total'] = df_export['sponsorship']
df_export.loc[df_export.type == 'rnd', 'sponsorship'] = np.nan

#Numberize and sum
df_export = remove_comma(df_export)
df_export = cleanup_number(df_export)
df_export = amounts_to_number(df_export)

export_acumulations(df_export, 'eisai')

,type,donations_grants,sponsorship,registration_fees,travel_accommodation,fees,related_expenses,total
0,hcp_amount,NaN,NaN,0.0,0.0,3875.0,0.0,3875.0
1,hco_amount,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,hcp_count,NaN,NaN,0.0,0.0,3.0,0.0,NaN
3,hco_count,0.0,0.0,0.0,0.0,0.0,0.0,NaN
4,rnd,NaN,NaN,NaN,NaN,NaN,NaN,26656.6


In [4]:
#write_to_excel(df_export, 'tmp.xlsx', open=True)